# Text parser for volume two

In [45]:
import os
with open("raw.txt", "r") as infile:
    data = infile.readlines()
    
    # Usable Parts of Section Two
    with open("Southern.txt", 'w') as outfile:
        outfile.writelines(data[3074:18607])
        
    # Usable Parts of Section Three
    with open("Northern.txt", 'w') as outfile:
        outfile.writelines(data[18611:39874])
        
    # Usable Parts of Section Four
    with open("Jiuliyang.txt", 'w') as outfile:
        outfile.writelines(data[39727:43643])

In [32]:
import re

def refine(file):
    # Function to process raw data for future further operation
    
    buffer = None
    
    with open(file, "r+") as f:
        buffer = f.read()
        f.seek(0)
        
        # Delete Watermarks
        buffer = re.sub("Kenneth Dean and Zhenman ZHENG - 978-90-47-44017-8", "", buffer).rstrip()
        buffer = re.sub("Downloaded from Brill.com02/18/2020 04:31:15AM", "", buffer).rstrip()
        buffer = re.sub("via National University of Singapore - YNC Library", "", buffer).rstrip()
        
        # Delete Headers
        buffer = re.sub("the southern irrigated plain 南洋水利系统", "", buffer).rstrip()
        buffer = re.sub("the northern irrigated plain 北洋水利系统", "", buffer).rstrip()
        buffer = re.sub("the jiuliyang irrigated plain 九里洋水利系统", "", buffer).rstrip()
        buffer = re.sub("part (one|two|three|four|five)", "", buffer).rstrip()
        
        # Delete Page Numbers
        buffer = re.sub("\n[0-9]+\n", "", buffer).rstrip()
        buffer = re.sub("\f(.*?)\n", "", buffer).rstrip()
        buffer = re.sub("\n\n(A|V)[0-9]+\n\n\n", "", buffer).rstrip()
        
        # Delete blank lines
        buffer = re.sub("\n+", " ", buffer).rstrip()
        
        # Add line breaks at each new entry
        buffer = re.sub("((A|V)[0-9]{3})", "\n\g<1>", buffer)

        f.truncate()        
        f.write(buffer)

In [46]:
refine("Southern.txt")
refine("Northern.txt")
refine("Jiuliyang.txt")



In [15]:
# Load Text and print them into seperate text files.
import json

villages = []


from pathlib import Path
Path("./Villages").mkdir(parents=True, exist_ok=True)

def createVillageFiles(path):

    with open(path) as infile:
        for line in infile:
            if line[0] == "V":
                uid = line[0:4]                
                buffer1 = line.split(" ", 3)
                buffer2 = buffer1[3].partition("2.")
                buffer3 = buffer2[2].partition("3.")
                buffer4 = buffer3[2].partition("4.")
                buffer5 = buffer4[2].partition("Birthday celebrations of the gods")
                
               
                entry = {
                    "UID": buffer1[0],
                    "Name_Pinyin": buffer1[1],
                    "Name_hanzi": buffer1[2],
                    "Region": path[:-4],
                    "Village_Settlement": buffer2[0][22:-1],
                    "Surname_Groups": buffer3[0][16:-1],
                    "Village_Temples": buffer4[0][17:-1],
                    "Rituals": {
                        "Yuanxiao processions": buffer5[0][31:-1],
                        "Birthday celebrations of the gods": buffer5[2][1:-2]
                    }
                }
                
                villages.append(entry)
                
                # Open and edit a new text file for each line
                with open("./Villages/"+uid+".json", "w") as outfile:
                    json.dump(entry, outfile, ensure_ascii=False)
                    outfile.close()
                    
                with open("./village_master.json", "w") as outfile:
                    json.dump(villages, outfile, ensure_ascii=False)
                    outfile.close()
                
        infile.close()
              

In [16]:
createVillageFiles("Southern.txt")
createVillageFiles("Northern.txt")
createVillageFiles("Jiuliyang.txt")

(' Village temples 1) Qianfeigong 钱妃宫 was originally called the Zhenhuimiao 贞惠庙, and later the Qianlinggong 钱灵宫. The temple was built during the Northern Song in Yuanfeng 7 (1083). The temple was renovated during the early Ming. In 1967 the temple was demolished, but it was rebuilt in 1985. The main god is Qianshi shengfei 钱氏圣妃. Secondary gods are Guangping Zhouwang 广平周王 and Taishan Kongwang 泰山孔王. The stone inscription composed in the Xianchun period (1265–1274) by Liu Kezhuang 刘克庄 entitled 协应钱夫人庙记 (Record of the Temple to Lady Qian of Beneficial Assistance) (Epigraphical Materials, 1995:54, No. 48) is about this temple (stele no longer extant). 2) Xinglongshê 兴隆社: The main gods are Zunzhu mingwang 尊主明王 and Houtu furen 后土夫人. ', '4.', ' Rituals: Yuanxiao processions On 1/10, the Li 李 and the Shi 施 surnames invite the incense burners of the shê altar to each of their homes to 行道 xingdao, carry out the Dao. On 1/13, the Wu 吴 and the Chen 陈 surnames do the same. On 1/14 the goddess Qianshi

(' Village temples 1) Yaotaijing Qingshuigong 瑶台境清水宫: Main hall: Middle altar: imperially enfeoffed Wanggong shenghou 敕封王公圣侯. Left altar: Dulue jiangjun 都略将军 (Yanggong taishi 杨公太师). Right altar: Shuntian shengmu 顺天圣母. Rear hall: Fazhu xianfei 法主仙妃, Zhaoling shengjun 昭灵圣君. Left side: Liuyuanshuai 刘元帅. Right side: Qiuyuanshuai 邱元帅. 2) Yutang sheshê 玉塘书社: Hall sign 匾额: Lishê weimin 立社为民. Main gods: Zunzhu mingwang 尊主明王, Houtu furen 后土夫人, Sima shengwang 司马圣王, Tianshang shengmu 天上圣母, Ligong zhangzhe 李公长者, Qianshi shengfei 钱氏圣妃. 3) Fuzhonggong 福中宫: Main god: Tiangong yuanshuai 田公元帅. 4) Zhongyitang 中一堂: Main god: Sanyi jiaozhu 三一教主. Secondary gods: Dichuan Lin Zhenming 嫡传林真明, Zaichuan Weng Wudao 再传翁吾道, Sanchuan Weng Yiyang 三传翁一阳, Sichuan Lin Zhengming 四传林正明, Wuchuan Zhang Jidao 五传张吉道, Guanyin dashi 观音大士, Jiuye 九爷. The SJCDCB, 1992:13 lists 290 initiates. ', '4.', ' Rituals: Yuanxiao processions On 1/4–5, Qingshuigong 清水宫 receives the gods. The first opera of the year, Touchangxi 头场戏, is perfo

(' Village temples 1) Fude dongshê 福德东社: Main god: Taiyi xiangu 太乙仙姑. 2) Houjiangjing Pukougong 垕江境浦口宫 is divided into Yongxing zhongshê 永兴中社 (the Wu 吴), Yongxing qianshê 永兴前社 (the Chen 陈), Yongxing houshê 永兴后社 (the Wu 吴), Yongxing yishê 永兴义社 (the Huang 黄 and the Chen 陈), Xinxing yishê 信兴义社 (the Wu 吴), Fude Jiangdong twelve shê ritual alliance dongshê 福德东社 (the Qi 祁), Fude xishê 福德西社 (the Lin 林), Huangxing sishê 黄兴祀社 (the Huang 黄), Taipingshê 太平社 (the Zhu 朱), Dongli jiahe 东里家社 (the Zheng 郑), Dongchun shangshê 东春上社 (miscellaneous surnames 杂姓), Xinan shoushê 新安寿社 (miscellaneous surnames 杂姓). There are 12 shê ritual associations in total. Apparently, there used to also be a Heshangshê “和尚社” (Buddhist monk ritual association), but now it no longer exists. Middle altar: Yuantian shangdi 元天上帝, Jiang meifei 江梅妃, Zunzhu mingwang 尊主明王, Houtu furen 后土夫人. Left altar: Jiang guojiu 江国舅, Wu zhenren 吴真人. Right altar: Tianshang shengmu 天上圣母, Guandi shengjun 关帝圣君. Jiang Meifei is a figure of local Puti

(' Village temples Kaipu dongshê 开莆东社 is dedicated to Zunzhu mingwang 尊主明王 and Houtu furen 后土夫人. Secondary gods: Lufu dashen 鲁府大神, Lufu furen 鲁府夫人, Lufu yima 鲁府姨妈, Aoshen 澳神. 2) Shanyuntang 善云堂 is dedicated to Lin Lonjiang 林龙江. The SJCDCB, 1992:14 claims that there are 1322 initiates. ', '4.', ' Rituals: Yuanxiao processions On 1/14, the gods of Kaipu dongshê 开莆东社 are taken to each (non-Christian) household to xingdao (carry out the Dao) and then go on procession within their territory. On 1/15 an overall Yuanxiao 总元宵 is held. The gods of Chongjiaogong 崇教宫 and Denglong zushê 登龙祖社 are invited to go on procession within the four shê alliance. Birthday celebrations of the gods 7/7: The birthday celebration of Lufu furen 鲁府夫人. Opera is performed. Daoist masters are invited to conduct rites. 8/14: The birthday celebration of Zunzhu mingwang 尊主明王 and Houtu furen 后土夫人. Opera is performed for two days. Daoist masters are invited to conduct rites. \n')
(' Rituals: Yuanxiao processions On 1/14, 

(' Village temples 1) Julongmiao 聚龙庙 or Beijidian 北极殿 is dedicated to Xuantian shangdi 玄天上帝. 2) Jumingci 聚明祠 was built in the late Ming dynasty. The temple is dedicated to Sanyi jiaozhu 三一教主. Other gods are Xuantan yuanshuai 玄坛 元帅 and Guansheng dijun 关圣帝君. The hall has over 100 Three in One initiates. ', '4.', ' Rituals: Yuanxiao processions On 1/11 the villagers go to Xiayuanhou 下院后 to the Yongxing zhongshê 永兴中社 to invite the incense burners from the shê-altar to be taken to each household to xingdao 行道. At night there is a lantern procession. On 1/12, the palm sedan chairs 棕轿 of gods go on a tour of inspection of the boundaries of the village, while Daoist rituals are conducted. Birthday celebrations of the gods On 3/3 Xuantian shangdi 玄天上帝 is celebrated with opera and Daoist rites. On 7/16 the Sanyi jiaozhu 三一教主 is celebrated with opera and Three in One rituals. \n')
(' Rituals: Yuanxiao processions On 1/11 the villagers go to Xiayuanhou 下院后 to the Yongxing zhongshê 永兴中社 to invite t

...

(' Village temples 1) Zifu dongshê 资福东社 belongs to the Fushan Chen 浮山陈: the main gods are Zunzhu mingwang 尊主明王, Houtu furen 后土 夫人, Sima shengwang 司马圣王, and Tiangong yuanshuai 田公元帅. Secondary gods include Fazhu xianfei 法主仙妃, Chenshi zhenjun 陈氏真君, Ciji zhenjun 慈济真君, Zhanggong shengjun 张公圣君, and Cigan xianfei 慈感仙妃. 2) Xing’an shushê 兴安书社 belongs to the Jiewai Chen 界外陈: the main gods are Zunzhu mingwang 尊主明王 and Houtu furen 后土夫人. Secondary gods include Fude zhengshen 福德正神. 3) Xinglongmiao 兴龙庙: The main god is Tiangong yuanshuai 田公元帅. Secondary gods include Yanggong taishi 杨公太师, Cuifu dama 崔府大妈, and Guanyin dashi 观音大士. 4) Fengchan shimiao 冯禅师庙: The main god is Feng Zhiri Chanshi 冯智 日禅师. Yushang two shê ritual alliance 5) Damafu 大妈府: The main god is Cuifu dama 崔府大妈. 6) Zongshengtang 宗圣堂: The main god is Sanyi jiaozhu 三一教主 Secondary gods include Puhua tianzun 普化天尊. 7) Ciguangsi Buddhist temple 慈光寺 was built in the early Republican period. (PTZJZCG, 1992:203) ', '4.', ' Rituals: Yuanxiao proce

('', '', '')
('', '', '')
(' Village temples 1) Chenghugong 澄湖宫: The main temple for Dingdun 顶墩, Liuqiao 柳桥 and Wudun 梧墩. The main god is Hangong yuanshuai 韩公元帅. Secondary gods include Ciji zhenjun 慈济真君, Qianshi shengfei 钱氏圣妃, Dutian yuanshuai 都天元帅, Fude zhengshen 福德正 神, Zhanggong shengjun 张公圣君, Heilong 黑龙, Baigou 白狗, Dage 大哥 Chen qingfu 陈庆福. 2) Xixun zhongshê 西浔中社: The main gods are Zunzhu mingwang 尊主 明王 and Houtu furen 后土夫人. Secondary gods include Fude zhengshen 福德正神. 3) Meicheng shengmiao 湄澄圣庙: The main gods are Tianshang shengmu 天上圣母 and Huangdu yuanshuai 黄都元帅. 4) Yuhuangdian 玉皇殿: The main god is Yuhuang dadi 玉皇大帝. Secondary gods include Tianshang wutian laomu 天上武天老母, Wangmu niangniang 王母娘娘, Tianshang wuji shengmu 天上无极圣母, Guanyin dashi 观音大士, Baogong 包公, Tongguan yuanshuai 统管元帅, and Taishang laojun 太上老君. ', '4.', ' Rituals: Yuanxiao processions On 1/14 the gods of Chenghugong 澄湖宫 go on a procession within the village. On 2/5 the rites of Toufu 头福 are celebrated by the Chen 陈. On 2/1

(' Rituals: Yuanxiao processions On 1/16 the gods of Gushanggong 古上宫 and Xingfushê 兴福社 are taken on a procession around the village to practice the Dao in every household. On 1/20 a common Yuanxiao festival 总元宵 is held. The gods of Zhaolinggong 昭灵宫 of Xiagongli 下宫里 go on a procession around the sevenfold ritual alliance. Opera is performed and Daoist ritual masters are invited to conduct rituals. These events are organized by Zhaolinggong, and funds are collected on a per capita basis. ', 'Birthday celebrations of the gods', ' 3/3: The birthday celebration of Xuantian shangdi 玄天上帝. 3/18: The birthday celebration of Lisan furen 李三夫人. 3/23: The birthday celebration of Tianshang shengmu 天上圣母. 4/26: The birthday celebration of Yanggong taishi 杨公太师. 5/13: The birthday celebration of Sima shengwang 司马圣王. 6/3: The birthday celebration of Zunzhu mingwang 尊主明王 and Houtu furen 后土夫人. 6/6: The birthday celebration of Fude zhengshen 福德正神. 8/14: The birthday celebration of Zhongtian dasheng 中天大圣. Th

(' Village temples 1) Longpo dongyishê 龙坡东义社: The main gods are Zunzhu mingwang 尊主明王, Houtu furen 后土夫人, Sima shengwang 司 马圣王, and Zhaoxian shengwang 昭显圣王. Secondary gods include Taibao laoye 太保老爷 and Taiwei yuanshuai 太尉元帅. 2) Ruiyun zumiao 瑞云祖庙 was built during the early Qing Dynasty. The main gods are Tiangong yuanshuai 田公元帅, Lufu dashen 鲁府大神, Lufu furen 鲁府夫人, and Lufu yima 鲁府姨妈. Secondary gods include Baiya jiangjun 白牙将军, Daye 大爷, Erye 二爷, and Zhong junye 中军爷. This temple is considered the founding temple of the cult of Tiangong yuanshuai in Putian and Xianyou, and many temples from these two counties come to present incense 进香 during Yuanxiao or on the god’s birthday. 3) Beiguan wudimiao 北关五帝庙 was rebuilt during the late Qing Dynasty. The main god is Yuhuang dadi 玉皇大帝. Secondary gods include Tiandijun qinshi 天地君亲师. 4) Yingxiangting 迎祥亭: The main god is Zhanggong shengjun 张公圣君, Secondary gods include Ciji zhenjun 慈济真君 and Wang xiansheng 王先生. 5) Tandu’an 坛渡庵: The main god is Shijia mo

PermissionError: [Errno 13] Permission denied: './village_master.json'